<a href="https://colab.research.google.com/github/bioai96/SKTFLYAI/blob/main/DL/Sonar_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_%EB%B0%8F_%EA%B2%80%EC%A6%9D(%EC%A4%91%EC%9A%94).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

In [2]:
!git clone https://github.com/taehojo/data.git

Cloning into 'data'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 21 (delta 3), reused 20 (delta 2), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [3]:
df = pd.read_csv('/content/data/sonar3.csv',header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,0
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,0
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,0
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,0
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,0


In [4]:
df[60].value_counts()

1    111
0     97
Name: 60, dtype: int64

In [6]:
x = df.iloc[:,0:60]
y = df.iloc[:,60]


In [ ]:
#모델 설정
model = Sequential()
model.add(Dense(24,input_dim = 60, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

#모델 컴파일
model.compile(loss ='binary_crossentropy', optimizer = 'adam',
              metrics = ['accuracy'])

#모델 실행
history = model.fit(x,y,epochs = 200 , batch_size=10)

In [8]:
# 여기서부터 의문 시작
# 과적합 - overfitting
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.3, shuffle = True)


In [10]:
score = model.evaluate(x_test, y_test)
print("Test accuracy : ", score[1])

2/2 [==============================] - 0s 8ms/step - loss: 0.0121 - accuracy: 1.0000
Test accuracy :  1.0


In [ ]:
#모델 설정
model = Sequential()
model.add(Dense(24,input_dim = 60, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

#모델 컴파일
model.compile(loss ='binary_crossentropy', optimizer = 'adam',
              metrics = ['accuracy'])

#모델 실행
history = model.fit(x_train, y_train, epochs = 200 , batch_size=10)


In [17]:
#모델을 테스트셋에 적용해 정확도 구하기
score = model.evaluate(x_test, y_test)
print('Test accuracy:', score[0])

2/2 [==============================] - 0s 9ms/step - loss: 0.7137 - accuracy: 0.7778
Test accuracy: 0.7137248516082764


## 모델 저장과 재사용

In [18]:
model.save('/content/data/my_model.hdf5')

In [25]:
from tensorflow.keras.models import Sequential, load_model
# 테스트를 위해 조금 전 만든 모델을 메모리에서 삭제
del model 
# 모델이 저장된 위치와 이름까지 적어 줍니다.
model = load_model('/content/data/my_model.hdf5')

score = model.evaluate(x_test, y_test)
print("Test accuracy :", score[1])


2/2 [==============================] - 0s 6ms/step - loss: 0.2253 - accuracy: 0.9512
Test accuracy : 0.9512194991111755


## k겹 교차 검증

In [43]:
from sklearn.model_selection import KFold
k = 5
kfold = KFold(n_splits = k, shuffle=True)
acc_score = []

def model_fn():
  #모델 설정
    model = Sequential()
    model.add(Dense(24,input_dim = 60, activation = 'relu'))
    model.add(Dense(10, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    return model

for train_index, test_index in kfold.split(x):
  x_train, x_test = x.iloc[train_index,:],x.iloc[test_index,:]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]
  
  model = model_fn()
  model.compile(loss='binary_crossentropy',optimizer = 'adam',
                metrics = ['accuracy'])
  history = model.fit(x_train,y_train,epochs = 200,batch_size=10,verbose=0)

  accuracy = model.evaluate(x_test,y_test)[1]
  acc_score.append(accuracy)

2/2 [==============================] - 0s 7ms/step - loss: 0.3379 - accuracy: 0.8780


In [44]:
# k겹 교차검증
avg_acc_score = sum(acc_score) / k

print("정확도: ",acc_score)
print("정확도 평균 : ",avg_acc_score)

정확도:  [0.8095238208770752, 0.8333333134651184, 0.8095238208770752, 0.9268292784690857, 0.8780487775802612]
정확도 평균 :  0.8514518022537232


정확도:  [0.8809523582458496, 0.8571428656578064, 0.8095238208770752, 0.8536585569381714, 0.8292682766914368, 0.8292682766914368]
정확도 평균 :  1.0119628310203552
